In [21]:
# Import required libraries
import pandas as pd
import dash
import jupyter_dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
import wget

In [22]:
from jupyter_dash import JupyterDash

In [23]:
spacex_launch_file=wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')


100% [................................................................................] 2476 / 2476

In [24]:
spacex_df=pd.read_csv(spacex_launch_file)
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [25]:
filtered_df = spacex_df[spacex_df['Launch Site'] == "CCAFS LC-40"].groupby(['Launch Site','class']).size().reset_index()
filtered_df.head()

,Launch Site,class,0
0,CCAFS LC-40,0,19
1,CCAFS LC-40,1,7


In [26]:
filtered_df = spacex_df[spacex_df['Launch Site']== "CCAFS LC-40"].groupby(['Launch Site','class']).size().reset_index(name='class count')

In [56]:
# Read the airline data into pandas dataframe
#spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                         style={'textAlign': 'center', 'color': '#503D36',
                                                'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Div(dcc.Dropdown(id='site-dropdown', 
                                               options=[
                                                    {'label':'All Sites','value': 'ALL'},
                                                    {'label':'CCAFS LC-40','value':'CCAFS LC-40'},
                                                    {'label':'KSC LC-39A','value':'KSC LC-39A'},
                                                    {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'},
                                                    {'label':'CCAFS SLC-40','value':'CCAFS SLC-40'}
                                                ],
                                                value='ALL',
                                                placeholder="Select a Launch Site here",
                                                searchable=True)),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                    # TASK 3: Add a slider to select payload range
                                html.Div(dcc.RangeSlider(id='payload-slider',
                                                    min=0, max=10000, step=1000,
                                                    marks={0:'0', 2500:'2500', 5000:'5000',
                                                           7500:'7500', 10000:'10000'},
                                                    value=[min_payload, max_payload])),
                                
                                html.Br(),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])


In [57]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output

@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site].groupby(['Launch Site','class']).size().reset_index(name='class count')
    if entered_site == 'ALL':
        piechart = px.pie(spacex_df, values='class', names='Launch Site', 
                     title='Total Success Launches by Site')
    else:
        piechart = px.pie(filtered_df, values='class count', names='class',
                      #title='Total Success Launches for Site {entered_site}'
                      title="Total Success Launches for {}".format(entered_site))
    return piechart
        # return the outcomes piechart for a selected site

@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id= 'payload-slider', component_property="value")])

def get_scatter_chart(entered_site, slider_range):
    low = slider_range[0]
    high = slider_range[1]
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(low,high)]
    filtered_df2 = filtered_df[filtered_df['Launch Site'] == entered_site]
    if entered_site == 'ALL':
        scatterchart = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                          title='Correlation between Payload and Success for all Sites')
    else:
        scatterchart = px.scatter(filtered_df2, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                          title=f'TCorrelation between Payload and Success for Site {entered_site}')
    return scatterchart

In [58]:
# Run the app
if __name__ == '__main__':
    # REVIEW8: Adding dev_tools_ui=False, dev_tools_props_check=False can prevent error appearing before calling callback function
    app.run_server()

Dash app running on http://127.0.0.1:8050/
